This notebook contains code to train a linear classifier on MNIST. At the end is a short exercise.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import numpy as np
import math

%matplotlib inline
import matplotlib.pyplot as plt

import scipy
import scipy.misc
import math
import PIL

import sys
import os
import re
import random
import glob

In [2]:
path = './images/'

sess = None
def ResetSession():
    tf.reset_default_graph()
    global sess
    if sess is not None: sess.close()
    sess = tf.InteractiveSession()
ResetSession()

In [3]:
NUM_CLASSES = 400
NUM_PIXELS = 200 * 200
TRAIN_STEPS = 1000
BATCH_SIZE = 100
LEARNING_RATE = 0.5

TEST_SIZE = 1000
TRAIN_SIZE = 4000

HIDDEN_1 = 60
HIDDEN_2 = 6

NUM_RES = 1

MAX_VAL = 255

whole_set = glob.glob(path + '*.png')
random.shuffle(whole_set)
test_set = whole_set[:TEST_SIZE]
train_set = whole_set[TEST_SIZE:TRAIN_SIZE + TEST_SIZE]
whole_set = []

train_labels = []
train_data = []
for i in train_set:
    regex_res = re.search('X([0-9]*\.?[0-9]+)_Y([0-9]*\.?[0-9]+)(_X([0-9]*\.?[0-9]+)_Y([0-9]*\.?[0-9]+))?.png', i)
    
    append_label = np.zeros((NUM_RES), dtype=np.float32)
    x_ind = (int(regex_res.group(1)))
    np.put(append_label, 0, x_ind)
    train_labels.append(append_label)
    
    img = scipy.misc.imread(i, flatten=False, mode='RGBA').astype(np.float)
    img = scipy.misc.imresize(img, 50)
    img = img[:,:,3]
    img.shape = (NUM_PIXELS)
    img = img / MAX_VAL
#     print (np.amin(img), np.amax(img))
#     train_data.append(img)
    train_data.append(img)
#     break
train_labels = np.array(train_labels)
train_data = np.array(train_data)
print ("done")

done


In [4]:
# mnist = input_data.read_data_sets('/tmp/data', one_hot=True)

# Placeholders for images and labels
# The first dimension is the batch size
# None means it can be of any length
# TensorFlow will infer it later
x = tf.placeholder(tf.float32, [None, NUM_PIXELS], name="pixels")
y_ = tf.placeholder(tf.float32, [None, NUM_RES], name="labels")

# Define the model
W_1 = tf.get_variable(shape=[NUM_PIXELS, HIDDEN_1], initializer=tf.contrib.layers.xavier_initializer(), name='w_1', regularizer=tf.contrib.layers.l2_regularizer(0.8))
b_1 = tf.get_variable(shape=[HIDDEN_1], initializer=tf.contrib.layers.xavier_initializer(), name='b_1', regularizer=tf.contrib.layers.l2_regularizer(0.8))
y_1 = tf.nn.relu(tf.matmul(x, W_1) + b_1)

W_2 = tf.get_variable(shape=[HIDDEN_1, HIDDEN_2], initializer=tf.contrib.layers.xavier_initializer(), name='w_2', regularizer=tf.contrib.layers.l2_regularizer(0.8))
b_2 = tf.get_variable(shape=[HIDDEN_2], initializer=tf.contrib.layers.xavier_initializer(), name='b_2', regularizer=tf.contrib.layers.l2_regularizer(0.8))
y_2 = tf.nn.relu(tf.matmul(y_1, W_2) + b_2)

W_3 = tf.get_variable(shape=[HIDDEN_2, NUM_RES], initializer=tf.contrib.layers.xavier_initializer(), name='w_3', regularizer=tf.contrib.layers.l2_regularizer(0.8))
b_3 = tf.get_variable(shape=[NUM_RES], initializer=tf.contrib.layers.xavier_initializer(), name='b_3', regularizer=tf.contrib.layers.l2_regularizer(0.8))
y = tf.matmul(y_2, W_3) + b_3

# Write a summary of the graph (before we add the loss and optimizer)
# Which will add a bunch of nodes automatically

# Define loss and optimizer
loss = tf.reduce_mean(tf.losses.mean_squared_error(predictions=y, labels=y_))
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)
tf.summary.scalar('loss', loss)

merged = tf.summary.merge_all()
sw = tf.summary.FileWriter('summaries/', graph=tf.get_default_graph())

# Initialize variables after the model is defined
sess.run(tf.global_variables_initializer())

# Train the model
for i in range(TRAIN_STEPS):
    sum_writer , _ , loss_val = sess.run([merged, train_step, loss], feed_dict={x: train_data, y_: train_labels})
    sw.add_summary(sum_writer, i)
#     if i % 100 == 0:
    print (i, "loss %f" % loss_val)
#         Weight = sess.run(W)
#         for j in range(NUM_CLASSES):
#             weight_0 = Weight[:, j]
#             weight_0.shape = (400, 400)
#             name = './weight_images/it_%d_class_%d.png' % (i, j)
#             scipy.misc.imsave(name, weight_0)
#             plt.imshow(weight_0, cmap=plt.cm.gray_r)
            
            

0 loss 53422.156250
1 loss 607824.625000
2 loss 54168.707031
3 loss 53447.335938
4 loss 53590.636719
5 loss 53622.320312
6 loss 53589.398438
7 loss 53522.746094
8 loss 53451.296875
9 loss 53356.597656
10 loss 53264.000000
11 loss 53166.281250
12 loss 53061.253906
13 loss 52949.859375
14 loss 52832.890625
15 loss 52711.023438
16 loss 52584.824219
17 loss 52454.804688
18 loss 52321.371094
19 loss 52184.914062
20 loss 52045.773438
21 loss 51904.218750
22 loss 51760.515625
23 loss 51614.898438
24 loss 51467.570312
25 loss 51318.714844
26 loss 51168.507812
27 loss 51017.085938
28 loss 50864.585938
29 loss 50711.136719
30 loss 50556.839844
31 loss 50401.812500
32 loss 50246.128906
33 loss 50089.886719
34 loss 49933.156250
35 loss 49776.007812
36 loss 49618.511719
37 loss 49460.710938
38 loss 49302.683594
39 loss 49144.464844
40 loss 48986.097656
41 loss 48827.632812
42 loss 48669.113281
43 loss 48510.562500
44 loss 48352.031250
45 loss 48193.539062
46 loss 48035.113281
47 loss 47876.777344
4

In [ ]:
test_data = []
test_labels = []
for i in test_set:
    regex_res = re.search('X([0-9]*\.?[0-9]+)_Y([0-9]*\.?[0-9]+)(_X([0-9]*\.?[0-9]+)_Y([0-9]*\.?[0-9]+))?.png', i)
    
    append_label = np.zeros((NUM_CLASSES), dtype=np.float32)
    x_ind = (int(regex_res.group(1)) - 1)
    np.put(append_label, x_ind, 1)
    test_labels.append(append_label)
    
    img = scipy.misc.imread(i, flatten=False, mode='RGBA').astype(np.float)
    img = img[:,:,3]
    img.shape = (NUM_PIXELS)
    img = img / MAX_VAL
#     print (np.amin(img), np.amax(img))
    test_data.append(img)
print ("done")

In [ ]:
correct_prediction = tf.equal(tf.argmax(y, 1),tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("Accuracy %f" % sess.run(accuracy, feed_dict={x: test_data,
                                  y_: test_labels}))

In [ ]:
print (len(test_labels), len(test_data))

In [ ]:
prediction = tf.argmax(y, 1)

def predict(idx):
    image = test_data[idx]
    return sess.run(prediction, feed_dict={x: [image]})

idx = 98
actual = np.argmax(test_labels[idx])
print ("Predicted: %d, Actual: %d" % (predict(idx), actual))
output = (test_data[idx].reshape((400,400)) * 255)
plt.imshow(output, cmap=plt.cm.gray_r)

In [ ]:
saver = tf.train.Saver()
saver.save(sess, './my_model/model', global_step=TRAIN_STEPS)